In [45]:
# os
import warnings
import time
import tqdm
import random
import string
import re
import csv
from pathlib import Path
from tqdm import tqdm
import argparse
import glob
import os
import json
import logging
from itertools import chain
from string import punctuation
import sentencepiece
from random import shuffle
from typing import List, Tuple

# модели
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, AutoModelForSequenceClassification

# nltk+torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.corpus import wordnet

warnings.filterwarnings("ignore")
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
RANDOM_STATE = 42
if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.manual_seed(RANDOM_STATE)
elif torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(RANDOM_STATE)
else:
    device = torch.device("cpu")

print(device)

cuda


In [3]:
full_text = """
“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.
"""

## 1. Получение текста и оценка его на генерацию по 100 бальной шкале

In [5]:
def is_generated_by_ai(paragraph):
    text_classifier = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=0)

    result = text_classifier(paragraph)
    confidence_threshold = 0.7
    label = result[0]['label']
    confidence = result[0]['score']
    if label == 'LABEL_1' and confidence >= confidence_threshold:
        return True, confidence
    return False, confidence

# Example usage
input_paragraph = "This is an example paragraph."
generated_by_ai, confidence = is_generated_by_ai(input_paragraph)

if generated_by_ai:
    print(f"The paragraph seems to be generated by AI with confidence {confidence}")
else:
    print(f"The paragraph seems to be written by a human with confidence {1 - confidence}.")

Device set to use cuda:0


The paragraph seems to be written by a human with confidence 0.7261924147605896.


## 2. Перефразирование текстов на английском с помощью T5

In [24]:
# функция для перефразирования текста c помощью T5 моделей
def paraphrase_t5(text, num_return_sequences=6, num_beams=12, min_new_tokens=60, max_new_tokens=350, no_repeat_ngram_size=4):

    system_prompt = (
        "Paraphrase this text in clearly other words."
        f"paraphrase: {text} </s>"
    )
    # заугркзка моделей
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl").to(device)
    model.eval()
    enc = tokenizer(system_prompt, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length,).to(device)

    # перефразирование
    with torch.inference_mode():
        out = model.generate(
            **enc,
            num_beams=num_beams,
            num_return_sequences=num_return_sequences,
            min_new_tokens=min_new_tokens,
            max_new_tokens=max_new_tokens,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=True,
            do_sample=True,
            top_k=120,
            top_p=0.95
        )

    do_sample=True,
    top_k=120,
    top_p=0.95,


    candidates = [tokenizer.decode(o, skip_special_tokens=True).strip() for o in out]
    smoothie = SmoothingFunction().method3
    ref_tokens = text.split()

    # отбор кандидатов
    scored = []
    for c in candidates:
        cand_tokens = c.split()
        bleu = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=smoothie)
        scored.append((c, bleu))

    scored = [pair for pair in scored if pair[1] > 0.3]

    scored.sort(key=lambda x: x[1], reverse=False)

    best = scored[0][0]
    return best, scored


In [25]:
best_paraphrase, ranked = paraphrase_t5(full_text, num_return_sequences=8, num_beams=16)

print("Original:\n", full_text, "\n")
print("Best paraphrase (by BLEU):\n", best_paraphrase, "\n")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Original:
 
“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.
 

Best paraphrase (by BLEU):
 This book is an autobiography of Dr. APJ Abdul Kalam, former President of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist. It reflects how simple living, dedication, strong will and hard work led to success. 



In [26]:
print(is_generated_by_ai(best_paraphrase))

Device set to use cuda:0


(False, 0.4561944305896759)


## 2. Перефразирование текстов на английском с помощью двойного перевода

In [12]:
# функция для перефразирования текста
def paraphrase_double_translation(text):

    # загрузка моделей машинного перевода
    src_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
    src_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru").to(device).eval()

    tgt_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
    tgt_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en").to(device).eval()
    # перевод текста с английского на русский
    input_ids = src_tokenizer.encode(text, return_tensors="pt").to(device)
    output_ids = src_model.generate(input_ids, max_length=300, num_beams=4, early_stopping=True)
    paraphrased_text_ru = src_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # перевод обратно с русского на английский
    input_ids = tgt_tokenizer.encode(paraphrased_text_ru, return_tensors="pt").to(device)
    output_ids = tgt_model.generate(input_ids, max_length=300, num_beams=4, early_stopping=True)
    paraphrased_text = tgt_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return paraphrased_text

In [13]:
original_text = "The quick brown fox jumps over the lazy dog."
paraphrased_text = paraphrase_double_translation(full_text)
print("Original text:", full_text)
print("Paraphrased text:", paraphrased_text)

Original text: 
“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.

Paraphrased text: ▪ Fire Wings: Abdul Kalam's autobiography is written by Dr. A J Abdul Kalam and Arun Tiwari, the autobiography of the former President of India, Dr. I-J Abdul Kalam, which sheds light on the boy's journey from Rameswaram to become a famous scientist, reflects how simple life, loyalty, strong will and hard work have led to success and how cultural unity affects people's lives.


In [15]:
print(is_generated_by_ai(paraphrased_text))

Device set to use cuda:0


(False, 0.4858350157737732)


## 2. Перефразирование текстов на русском с помощью T5

In [28]:
full_text = "Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены. Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции."

In [17]:
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model.to(device)
model.eval()

def paraphrase_rus(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)



config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [19]:
pere_rus = paraphrase_rus(full_text)
pere_rus

'Многие полагают, что интеллектуальный интеллект способен заменить человека полностью, но в действительности возможности его ограничены. У машины есть умение анализировать данных и прогнозировать, но они не могут понять контекст настолько глубоко, сколько человек делает. Для настоящего мышления нужны не только логические, но опыт, эмоции и инстинкты.'

In [20]:
print(is_generated_by_ai(pere_rus))

Device set to use cuda:0


(False, 0.558468759059906)


## 2. Перефразирование текстов на русском с помощью двойного перевода

In [29]:
# функция для перефразирования текста
def paraphrase_double_translation(text):

    # загрузка моделей машинного перевода
    src_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
    src_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en").to(device).eval()

    tgt_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
    tgt_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru").to(device).eval()
    # перевод текста с русского на английский
    input_ids = src_tokenizer.encode(text, return_tensors="pt").to(device)
    output_ids = src_model.generate(input_ids, max_length=300, num_beams=4, early_stopping=True)
    paraphrased_text_ru = src_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # перевод обратно с англиского на русский
    input_ids = tgt_tokenizer.encode(paraphrased_text_ru, return_tensors="pt").to(device)
    output_ids = tgt_model.generate(input_ids, max_length=300, num_beams=4, early_stopping=True)
    paraphrased_text = tgt_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return paraphrased_text

In [31]:
pere_rus_d = paraphrase_double_translation(full_text)
pere_rus_d

'Многие считают, что искусственный интеллект может полностью заменить людей, но на самом деле его возможности ограничены. Машины могут анализировать данные и делать прогнозы, но они не могут понять контекст так глубоко, как человек. Реальное мышление требует не только логики, но и опыта, эмоций и интуиции.'

In [32]:
print(is_generated_by_ai(pere_rus_d))

Device set to use cuda:0


(False, 0.5254822969436646)


## 3. Камуфлирование текста

In [34]:
# знаки перпинания
def modify_syntax(text):
    modified_text = text

    # генерация случайных порогов вероятности
    add_spaces_around_punctuation = random.uniform(0.74, 0.9)
    remove_extra_punctuation = random.uniform(0.6, 0.8)
    replace_question_exclamation = random.uniform(0.5, 0.74)
    normalize_ellipsis = random.uniform(0.7, 0.9)
    add_spaces_around_non_alphanumeric = random.uniform(0.75, 1.0)

    # добавление пробелов вокруг знаков препинания
    if random.random() < add_spaces_around_punctuation:
        modified_text = re.sub(r'([,.\?!])', r' \1 ', modified_text)
        if random.random() < 0.5:
            modified_text = re.sub(r' +', ' ', modified_text)

    # удаление лишних знаков препинания
    if random.random() < remove_extra_punctuation:
        modified_text = re.sub(r'[,\.]+(?=[\)\]"])', '', modified_text)
        if random.random() < 0.3:
            modified_text = re.sub(r'[,\.]+', random.choice([',', '.', '']), modified_text)

    # замена вопросительного знака с восклицательным
    if random.random() < replace_question_exclamation:
        modified_text = re.sub(r'\?!', random.choice(['?', '!']), modified_text)

    # нормализация использования многоточий
    if random.random() < normalize_ellipsis:
        modified_text = re.sub(r'\.{2,}', '...', modified_text)
        if random.random() < 0.4:
            modified_text = re.sub(r'\.{3}', random.choice(['...', '...']), modified_text)

    # добавление пробелов вокруг непечатных символов
    if random.random() < add_spaces_around_non_alphanumeric:
        modified_text = re.sub(r'([^a-zA-Z0-9])', r' \1 ', modified_text)
        if random.random() < 0.6:
            modified_text = re.sub(r' +', ' ', modified_text)

    # удаление лишних пробелов
    modified_text = re.sub(r'\s+', ' ', modified_text)

    return modified_text

modify_syntax(full_text)

'Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены, Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек, Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции,'

In [37]:
# изменениие длины предложений
import re
def modify_sentence_length(text):
    sentences = re.split(r'[.!?]+', text)

    modified_sentences = []
    for sentence in sentences:
        if len(sentence.split()) > 20:
            short_sentences = re.split(r'[,;]+', sentence)
            modified_sentences.extend(short_sentences)
        else:
            if len(modified_sentences) > 0 and len(modified_sentences[-1].split()) < 10:
                modified_sentences[-1] += ' ' + sentence.strip()
            else:
                modified_sentences.append(sentence.strip())

    modified_text = '. '.join(modified_sentences) + '.'

    return modified_text


modified_text = modify_sentence_length(full_text)

print("Оригинальный текст:")
print(full_text)
print("\nМодифицированный текст:")
print(modified_text)

Оригинальный текст:
Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены. Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции.

Модифицированный текст:
Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены. Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции. .


In [39]:
# перестановка слов
def modify_word_order(text):
    sentences = re.split(r'[.!?]+', text)

    modified_sentences = []
    for sentence in sentences:
        words = sentence.split()

        if len(words) > 2:
            middle_words = words[1:-1]
            shuffle(middle_words)

            modified_sentence = ' '.join([words[0]] + middle_words + [words[-1]])
        else:
            modified_sentence = ' '.join(words)

        modified_sentences.append(modified_sentence)

    modified_text = '. '.join(modified_sentences) + '.'

    return modified_text


modified_text = modify_word_order(full_text)

print("Оригинальный текст:")
print(full_text)
print("\nМодифицированный текст:")
print(modified_text)

Оригинальный текст:
Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены. Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции.

Модифицированный текст:
Многие его что но человека, считают, самом искусственный способен деле заменить на интеллект полностью возможности ограничены. Машины это они анализировать прогнозы, однако данные делает не и контекст понять способны как глубоко, так умеют делать человек. Настоящее требует и только эмоций но не опыта, логики, мышление и интуиции. .


In [42]:
# замена пассивных конструкций на активные
def activate_passive_sentences(text):
    active_text = re.sub(r'(\w+) was (\w+)', r'\2 \1', text)
    return active_text

# добавление эмоциональных/оценочных элементов
def add_emotional_elements(text):
    emotional_words = [
        'весело', 'радостно', 'волнушкой', 'смеяться',
        'возбужденно', 'вдохновляюще', 'интригующе', 'взрывно',
        'трепетом', 'волнением', 'страстно', 'горячо', 'влюбленно'
    ]

    for word in emotional_words:
        text = re.sub(r'\bкроме того\b', f'{word} ', text)

    return text

# изменение стиля изложения на более разговорный
    conversational_phrases = {
        'формальный': 'разговорный',
        'предложения': 'бабочки',
        'легче': 'просто',
        'воспринимаются': 'понимаются',
        'включение': 'добавление',
        'живым': 'активным',
        'выразительным': 'выражительным',
        'кроме того': 'еще',
        'также': 'а еще',
        'дополнительно': 'и так далее',
        'конечно': 'ну конечно',
        'действительно': 'правильно',
        'по сути': 'в общем',
        'в основном': 'в основном'
    }

    for key, value in conversational_phrases.items():
        text = text.replace(key, value)

    return text


# применяем функции к тексту
activated_text = activate_passive_sentences(full_text)
emotional_text = add_emotional_elements(activated_text)

print("Оригинальный текст:")
print(full_text)
print("\nМодифицированный текст:")
print(emotional_text)

Оригинальный текст:
Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены. Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции.

Модифицированный текст:
Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены. Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции.


In [44]:
# случайные перестановки букв в словах
def scramble_words_with_control(text, probability):
    words = re.findall(r'\b\w+\b', text)
    for i, word in enumerate(words):
        if random.random() < probability:
            scrambled_word = ''.join(random.sample(word, len(word)))
            text = text.replace(word, scrambled_word)
    return text

# пример использования функции
probability=0.1
controlled_scrambled_text = scramble_words_with_control(full_text, probability=probability)

print("Оригинальный текст:")
print(full_text)
print("\nМодифицированный текст:")
print(controlled_scrambled_text)

Оригинальный текст:
Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены. Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции.

Модифицированный текст:
Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его миоовсожнтз ограничены. Машины умеют анализировать данные и делать прогнозы, однако ино не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и ыоатп, эмоций и интуиции.


In [46]:
# замена на синонимы
def find_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def replace_synonyms(text):
    words = text.split()
    new_words = []
    for word in words:
        if word.lower() in wordnet.words():
            synonyms = find_synonyms(word)
            if len(synonyms) > 0:
                new_word = random.choice(synonyms)
                new_words.append(new_word)
            else:
                new_words.append(word)
        else:
            new_words.append(word)
    return ' '.join(new_words)

# пример использования
full_text = """
“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.
"""
print("Оригинальный текст:")
print(full_text)
print("\nМодифицированный текст:")
print(replace_synonyms(full_text))

Оригинальный текст:

“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.


Модифицированный текст:
“Wings of Fire: AN autobiography of Abdul Kalam” is publish by doc APJ Abdul Kalam and Arun Tiwari. This book is Associate_in_Nursing autobiography of MD APJ Abdul Kalam, early Chief_Executive of India. This write_up sheds light on the travel of A Young boy from Rameswaram to go a noted scientist. IT reflects how wide-eyed living, dedication, hard bequeath and severely work guide to success. information_technology likewise shows how cultural one impacts the lives of individuals.


## 4. Метрика сравнения сгенеренного текста и исходного

In [22]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-base-cased-dp-paraphrase-detection")

def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
    return proba[0] # p(non-paraphrase), p(paraphrase)

print(compare_texts('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))
print(compare_texts('Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась'))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[0.7056226  0.29437745]
[0.16524385 0.83475614]


In [27]:
full_text = """
“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.
"""
print("T5:", compare_texts(full_text, best_paraphrase))
print("double:", compare_texts(full_text, paraphrased_text))


T5: [0.03036341 0.96963656]
double: [0.03524494 0.96475506]


In [33]:
full_text = "Многие считают, что искусственный интеллект способен полностью заменить человека, но на самом деле его возможности ограничены. Машины умеют анализировать данные и делать прогнозы, однако они не способны понять контекст так глубоко, как это делает человек. Настоящее мышление требует не только логики, но и опыта, эмоций и интуиции."
print("T5:", compare_texts(full_text, pere_rus))
print("double:", compare_texts(full_text, pere_rus_d))


T5: [0.04288651 0.9571135 ]
double: [0.03636042 0.9636396 ]
